<a href="https://colab.research.google.com/github/JD32919/EDA/blob/main/Hash_table_1M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 # Implementación de MurmurHash3 para cadenas de texto
def murmurhash3_32(key, seed=0):
    key = key.encode('utf-8')
    c1 = 0xcc9e2d51
    c2 = 0x1b873593
    r1 = 15
    r2 = 13
    m = 5
    n = 0xe6546b64
    hash_value = seed

    # Procesar bloques de 4 bytes
    for i in range(0, len(key) // 4 * 4, 4):
        k = key[i] | (key[i + 1] << 8) | (key[i + 2] << 16) | (key[i + 3] << 24)
        k = (k * c1) & 0xFFFFFFFF
        k = (k << r1 | k >> (32 - r1)) & 0xFFFFFFFF
        k = (k * c2) & 0xFFFFFFFF

        hash_value ^= k
        hash_value = (hash_value << r2 | hash_value >> (32 - r2)) & 0xFFFFFFFF
        hash_value = (hash_value * m + n) & 0xFFFFFFFF

    # Procesar los últimos bytes del mensaje
    tail = len(key) & 3
    if tail == 3:
        k1 = (key[len(key) - 3] << 16) | (key[len(key) - 2] << 8) | key[len(key) - 1]
    elif tail == 2:
        k1 = (key[len(key) - 2] << 8) | key[len(key) - 1]
    elif tail == 1:
        k1 = key[len(key) - 1]
    else:
        k1 = 0

    if tail != 0:
        k1 = (k1 * c1) & 0xFFFFFFFF
        k1 = (k1 << r1 | k1 >> (32 - r1)) & 0xFFFFFFFF
        k1 = (k1 * c2) & 0xFFFFFFFF
        hash_value ^= k1

    # Finalización
    hash_value ^= len(key)
    hash_value ^= hash_value >> 16
    hash_value = (hash_value * 0x85ebca6b) & 0xFFFFFFFF
    hash_value ^= hash_value >> 13
    hash_value = (hash_value * 0xc2b2ae35) & 0xFFFFFFFF
    hash_value ^= hash_value >> 16
    return hash_value

# Implementación de la clase HashTable con encadenamiento
class HashTable:
    def __init__(self, size=1_000_000):
        self.size = size
        self.table = [[] for _ in range(size)]

    def hash_function(self, key):
        # Usar MurmurHash3 para calcular el índice
        return murmurhash3_32(key) % self.size

    def insert(self, key, value):
        index = self.hash_function(key)
        # Actualizar valor si la clave ya existe
        for i, (k, v) in enumerate(self.table[index]):
            if k == key:
                self.table[index][i] = (key, value)
                return
        # Insertar nueva clave si no existe
        self.table[index].append((key, value))

    def search(self, key):
        index = self.hash_function(key)
        # Buscar la clave en la lista enlazada
        for k, v in self.table[index]:
            if k == key:
                return v
        return None  # Si la clave no se encuentra

    def delete(self, key):
        index = self.hash_function(key)
        # Eliminar el par clave-valor si existe
        for i, (k, v) in enumerate(self.table[index]):
            if k == key:
                del self.table[index][i]
                return True
        return False  # Si no se encontró la clave

    def display(self):
        # Mostrar el contenido de la tabla hash
        for i, bucket in enumerate(self.table):
            if bucket:  # Mostrar solo posiciones no vacías
                print(f"Índice {i}: {bucket}")

# Ejemplo de uso de la tabla hash
if __name__ == "__main__":
    # Crear una tabla hash con tamaño de exactamente 1 millón
    hash_table = HashTable(size=1_000_000)

    # Insertar 20 datos de demostración en la tabla
    keys = [f"clave{i}" for i in range(1, 21)]
    values = [i * 100 for i in range(1, 21)]

    for key, value in zip(keys, values):
        hash_table.insert(key, value)

    # Mostrar el contenido de la tabla hash
    hash_table.display()

    # Buscar algunos datos en la tabla
    for key in ["clave1", "clave10", "clave20"]:
        print(f"Valor de '{key}':", hash_table.search(key))

    # Eliminar un dato y buscarlo de nuevo
    hash_table.delete("clave10")
    print("Valor de 'clave10' después de eliminar:", hash_table.search("clave10"))

